## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
print(application_df.shape)
application_df.head()


(34299, 12)


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN"])
print(application_df.shape)
application_df.head()

(34299, 11)


,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column. 
application_df.nunique(axis=0)

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_type_count = application_df["APPLICATION_TYPE"].value_counts()
app_type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced   ## grouping minor categories together to make it stronger (safer), higher value count will impact the way the model learns
# use the variable name `application_types_to_replace`                          ## binning work if there're less than 10 unique values in a column so we could do dummies 
application_types_to_replace = list(app_type_count[app_type_count < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classifications_count = application_df["CLASSIFICATION"].value_counts()
classifications_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
print(f"There are {len(classifications_count[classifications_count > 1])} different CLASSIFICATIONS with value count greater than 1.")
classifications_count[classifications_count > 1]

There are 45 different CLASSIFICATIONS with value count greater than 1.


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classifications_count[classifications_count < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`   # get dummies ignores the columns with numerical values
application_df_ = pd.get_dummies(application_df)  
application_df_.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_1 DAY RANCH RESCUE AND RURAL OKLAHOMA ANIMAL RESOURCE INC,NAME_100 BLACK MEN OF AMERICA,NAME_100 BLACK MEN OF MEMPHIS INC,NAME_100 BLACK MEN OF WEST GEORGIA INC,NAME_1150 WEBSTER STREET INC,NAME_116TH CAVALRY REGIMENT CHAPTER OF THE US CAVALRY & ARMOR ASSOCIATION,NAME_13TH BOMB SQUADRON ASSOCIATION,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
# Separate the y variable, the target
y = application_df_["IS_SUCCESSFUL"]

# Separate the X variable, the features
X = application_df_.drop(columns = "IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
# Create a StandardScaler instances   
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Compile, Train and Evaluate the Model

## OPTIMIZATION ATTMEPT #1

- Dropping more or fewer columns. (Keeping the `NAME` column from the orignial data)

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0]) # The length of X variable - the features.
hidden_layer_1 = 80
hidden_layer_2 = 30

nn_1 = tf.keras.models.Sequential()

# First hidden layer
nn_1.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="relu", input_dim=input_features))
# Second hidden layer
nn_1.add(tf.keras.layers.Dense(units=hidden_layer_2, activation="relu"))

# Output layer
nn_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
print(nn_1.summary())
# -------------------------------------------------------------------------
# Compile the model
nn_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_1.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
print("-------------------------------------------------------------------------")
model_loss, model_accuracy = nn_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                1568960   
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 1571421 (5.99 MB)
Trainable params: 1571421 (5.99 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None

Epoch 1/100


804/804 [==============================] - 15s 17ms/step - loss: 0.4928 - accuracy: 0.7610
Epoch 2/100
804/804 [==============================] - 13s 17ms/step - loss: 0.1256 - accuracy: 0.9540
Epoch 3/100
804/804 [==============================] - 13s 16ms/ste

804/804 [==============================] - 13s 16ms/step - loss: 0.0770 - accuracy: 0.9685
Epoch 64/100
804/804 [==============================] - 13s 16ms/step - loss: 0.0774 - accuracy: 0.9681
Epoch 65/100
804/804 [==============================] - 13s 16ms/step - loss: 0.0768 - accuracy: 0.9684
Epoch 66/100
804/804 [==============================] - 13s 16ms/step - loss: 0.0771 - accuracy: 0.9686
Epoch 67/100
804/804 [==============================] - 13s 16ms/step - loss: 0.0768 - accuracy: 0.9682
Epoch 68/100
804/804 [==============================] - 13s 16ms/step - loss: 0.0765 - accuracy: 0.9682
Epoch 69/100
804/804 [==============================] - 13s 16ms/step - loss: 0.0768 - accuracy: 0.9687
Epoch 70/100
804/804 [==============================] - 13s 16ms/step - loss: 0.0769 - accuracy: 0.9684
Epoch 71/100
804/804 [==============================] - 13s 16ms/step - loss: 0.0766 - accuracy: 0.9683
Epoch 72/100
804/804 [==============================] - 13s 16ms/step - loss:

## OPTIMIZATION ATTEMPT #2

- Add more neurons to a hidden layer (Increasing hidden layer 1 nodes from 80 to 100, increasing hidden layer 2 nodes from 30 to 50)
- Add more hidden layers. (Adding a third hidden layer with 25 nodes)

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
hidden_layer_1 = 100
hidden_layer_2 = 50
hidden_layer_3 = 25

nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="relu", input_dim=input_features))
# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_layer_2, activation="relu"))
# Third hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_layer_3, activation="relu"))

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
print(nn_2.summary())
# -----------------------------------------------------------------------------------
# Compile the model
nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_2.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
print("-------------------------------------------------------------------------")
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               1961200   
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 25)                1275      
                                                                 
 dense_6 (Dense)             (None, 1)                 26        
                                                                 
Total params: 1967551 (7.51 MB)
Trainable params: 1967551 (7.51 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/100
804/804 [==============================] - 19s 21ms/step - loss: 0.4764 - accuracy: 0.7742
Epoch 2/100
804/804 [===================

804/804 [==============================] - 16s 20ms/step - loss: 0.0774 - accuracy: 0.9682
Epoch 71/100
804/804 [==============================] - 16s 20ms/step - loss: 0.0781 - accuracy: 0.9678
Epoch 72/100
804/804 [==============================] - 16s 20ms/step - loss: 0.0770 - accuracy: 0.9685
Epoch 73/100
804/804 [==============================] - 16s 20ms/step - loss: 0.0763 - accuracy: 0.9688
Epoch 74/100
804/804 [==============================] - 16s 20ms/step - loss: 0.0764 - accuracy: 0.9683
Epoch 75/100
804/804 [==============================] - 16s 20ms/step - loss: 0.0766 - accuracy: 0.9682
Epoch 76/100
804/804 [==============================] - 16s 20ms/step - loss: 0.0763 - accuracy: 0.9682
Epoch 77/100
804/804 [==============================] - 16s 20ms/step - loss: 0.0763 - accuracy: 0.9685
Epoch 78/100
804/804 [==============================] - 16s 20ms/step - loss: 0.0765 - accuracy: 0.9686
Epoch 79/100
804/804 [==============================] - 16s 20ms/step - loss:

## OPTIMIZATION ATTEMPT #3

- Use different activation functions for the hidden layers.(Change activation function from relu to sigoid for all hidden layers)
- Add or reduce the number of epochs to the training regimen. (Reduce the number of epoches from 100 to 50)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
hidden_layer_1 = 80
hidden_layer_2 = 30

nn_3 = tf.keras.models.Sequential()

# First hidden layer
nn_3.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="sigmoid", input_dim=input_features))
# Second hidden layer
nn_3.add(tf.keras.layers.Dense(units=hidden_layer_2, activation="sigmoid"))

# Output layer
nn_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
print(nn_3.summary())
# -----------------------------------------------------------------------------------
# Compile the model
nn_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_3.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
print("-------------------------------------------------------------------------")
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                1568960   
                                                                 
 dense_8 (Dense)             (None, 30)                2430      
                                                                 
 dense_9 (Dense)             (None, 1)                 31        
                                                                 
Total params: 1571421 (5.99 MB)
Trainable params: 1571421 (5.99 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/50
804/804 [==============================] - 15s 17ms/step - loss: 0.5351 - accuracy: 0.7371
Epoch 2/50
804/804 [==============================] - 13s 16ms/step - loss: 0.1870 - accuracy: 0.9348
Epoch 3/50
804/804 [==============================] - 13s 16ms/step - l

In [15]:
# Export our model to HDF5 file
nn_3.save("AlphabetSoupCharity_Optimization.h5")     

C:\Users\alaa3\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
